# Validation of link budget model
For verification against budget in SMAD, Table 13-13, page 567 (3rd edition), Telemetry and Data column
Note, that there seems to be a double counting of $L_{pr}$ in the equations

In [4]:
from datastructures.api import Model, adda
from constants import k, c
from numpy import pi
from math import log10
import sympy as sp
from compute import Par
from datastructures.postprocess import print_outputs_static

In [47]:
dBtoLinear = lambda db: 10**(db/10)
lineartodB = lambda l: 10*log10(l)
# Telemetry and Data column - Command column
S = Par('S', 2831, 'km')   # 2831
P = Par('P', 1, 'W')      # 1
theta_t = Par(r'\theta_t', 2) # 2
#D_t = Par('D_t', 0.3, 'm') # 5.3
D_r = Par('D_r', 0.07, 'm') # 0.07
L = Par('L', dBtoLinear(1+0.1+0.3+2.6)) #L_l+L_pt+L_a+Lpr 
T_s = Par('T_s', 614, 'K') # 614
eta = Par(r'\eta', 0.55)   # assumed but not given explicitely in table
f = Par('f', 2, 'GHz')   # 2 
R = Par('R', 100, unit='bit/s') #100 kbps

# Link budget equations

In [48]:
model = Model()
m = model.root
λ_c = adda(m, r'\lambda_c', c/f, unit='cm')
#G_t = adda(m, 'G_t', eta*(pi*D_t/λ_c)**2)
G_pt = adda(m, r'G_{pt}', dBtoLinear(44.3-10*sp.log(theta_t*theta_t, 10))) # this is the actual model used in the book
D_t = adda(m, 'D_T', λ_c*(G_pt/eta)**0.5/pi, unit='m')
G_rp = adda(m, r'G_{rp}', eta*(pi*D_r/λ_c)**2)
L_s = adda(m, 'L_s', (λ_c/(4*pi*S))**2, unit='')
EN = adda(m, 'EN', P*G_rp*G_pt/(L*k*T_s*R)*L_s, unit='')

### Check values

In [58]:
lineartodB(G_rp.varval)

0.755761295901285

In [57]:
lineartodB(G_rp.varval)-2.6

-1.8442387040987152

In [50]:
lineartodB(L_s.varval) #L_s in dB

-167.53017081453504

### Check EIRP

In [56]:
lineartodB(G_pt.varval)-0.1+lineartodB(1)-1

37.17940008672037

In [63]:
37.2-0.3-0.1-167.5-1.9+228.6-10*log10(T_s.varval)-10*log10(100)-2.6

45.518316288588316

In [53]:
lineartodB(G_pt.varval)+lineartodB(20)-lineartodB(L.varval)+lineartodB(L_s.varval)+39-lineartodB(k.varval)-10*log10(T_s.varval)-10*log10(85e6)

40.182824943367876

It seems like the Lpt is double counted in SMAD

In [64]:
lineartodB(EN.varval)

48.22247553977231

## Results

In [65]:
print_outputs_static(model).style.hide(axis="columns").hide(axis="index")

$$\lambda_c$$,14.95,cm
$$G_{pt}$$,6728.837,
$$D_T$$,5.264,m
$$G_{rp}$$,1.19,
$$L_s$$,1.766×10⁻¹⁷,
$$EN$$,6.641×10⁴,


# Other interesting calculations

Fig. 13-6. $G_t=5, \theta=75; G_t=125, \theta=15$

In [76]:
27000/(75*75)

4.8

In [78]:
27000/(15*15) #it's 5 off

120.0